In [19]:
import pandas as pd

In [20]:
def read_csv_files(file_paths):
    """
    本函数用于读取所有的CSV文件并存入多个Dataframe中。
    :param file_paths: 一个或多个文件路径 
    :return: 一个或多个Dataframe
    """
    dfs = []
    for i, path in enumerate(file_paths):
        dfs.append(pd.read_csv(path, index_col=0).stack())
    return dfs

In [21]:
def determine_status_sign(status):
    """
    本函数用于处理异常的股票，给股票打上标签，如果成分异常，则打上-1；如果成分正常或者由异常变为正常，则打上1。
    1	ST
    3	PT
    2	撤销ST
    5	*ST
    7	撤消*ST并实行ST
    8	从ST变为*ST
    12	叠加ST
    6	撤销*ST
    9	退市整理期
    4	撤销PT
    13	撤销叠加ST
    14	叠加*ST
    15	撤销叠加*ST
    10	高风险警示
    11	撤销高风险警示
    :param status: 成分异常的表中有一列名为status，根据status的值分别给股票打上标签
    :return: 返回一个具有标签的Dataframe
    """
    if status in [1, 5, 7, 8, 9, 12, 13, 14]:
        return -1
    elif status in [2, 3, 4, 6]:
        return 1
    else:
        return None

In [22]:
def merge_dataframes(list_of_dfs):
    # 使用 concat 方法按列合并 DataFrame
    merged_df = pd.concat(list_of_dfs, axis=1)
    return merged_df

In [23]:
file_paths = [
    'data/昨收价.csv',  # 昨收价
    'data/收盘价.csv',  # 收盘价
    'data/昨开盘.csv',  # 做开盘价
    'data/开盘价.csv',  # 开盘价
    'data/涨停价.csv',  # 涨停价
    'data/涨跌幅.csv'  # 涨跌幅
#     '筛选涨跌幅.csv'
]

df_pre_close, df_close, df_pre_open, df_open, df_stopin, df_ud = read_csv_files(file_paths)

In [24]:
df_st = pd.read_csv('data/st表.csv', index_col=0)
# 利用-1和1区别status数值，并存在status sign中
df_st['sign'] = df_st['SpecialTradeType'].apply(determine_status_sign)
# 删除inner_id和status，只保留日期day，股票名字id和status sign
df_st = df_st.drop(df_st.columns[[0, 2]], axis=1)
# 将两列打成一行z一列，index为day列，行为id列，数值则是status sign
df_ST_pivot = df_st.pivot(index='SpecialTradeTime', columns='SecuCode', values='sign')
df_ST_pivot.index = pd.to_datetime(df_ST_pivot.index)
df_ST_pivot = df_ST_pivot.resample('D').asfreq()
# 向后传播填写NaN，再把其他的NaN填充为1
df_ST_pivot = df_ST_pivot.ffill().fillna(1)
# 补充日期，从1988.4.28到2023.12.18
date_range = pd.date_range(start='1988-04-28', end='2023-12-29', freq='D')
# 填充日期，如果是全量数据就不需要这个数据
df_ST_pivot = df_ST_pivot.reindex(date_range)
df_ST_pivot = df_ST_pivot.ffill().fillna(1)
df_ST_pivot.index.name = 'day'
df_ST_pivot.rename(columns={'0': 'sign'}, inplace=True)
# 将dataframe变成mutindex，方便之后merge
df_ST_stack = pd.DataFrame(df_ST_pivot.stack())
df_ST_stack.columns = ['sign']
df_ST_stack.index.names = None, None

In [25]:
df = pd.read_csv('data/昨总股本.csv', index_col=0)
df = df.stack()
df = pd.DataFrame(df)
df.columns = ['pre_number']

In [26]:
# 刚上市的股票没有资格进入成分股，只有非涨停日第三天才能选入成分股
df1 = pd.read_csv('data/购买信息.csv', index_col=0)
df1 = df1.stack()
df1 = pd.DataFrame(df1)
df1.columns = ['tradable']

In [27]:
merge_df = merge_dataframes([df_pre_close, df_close, df_pre_open, df_open, df_stopin, df_ud])
merge_df.columns = ['pre_close', 'close','pre_open', 'open', 'stopin', 'ud']

merge_df = pd.merge(merge_df, df, left_index=True, right_index=True, how='left')
merge_df = pd.merge(merge_df, df1, left_index=True, right_index=True, how='left')

merge_df['pre_close'] = merge_df.pre_close.astype(float).fillna(0)
merge_df['pre_number'] = merge_df.pre_number.astype(float).fillna(0)
merge_df['value'] = merge_df['pre_close'] * merge_df['pre_number']
merge_df = merge_df.fillna(0)

merge_df = pd.merge(merge_df, df_ST_stack, left_index=True, right_index=True, how='left')
merge_df = merge_df.fillna(1)
merge_df.index.names = ['day', 'id']

In [28]:
print(merge_df)

                      pre_close  close  pre_open   open  stopin      ud  \
day        id                                                             
2000-01-04 000001.SZ      17.45  18.29     17.47  17.50   19.20  4.8138   
           000002.SZ       9.75  10.30      9.70   9.80   10.73  5.6410   
           000003.SZ       5.47   5.74      5.50   5.48    6.02  4.9360   
           000004.SZ       8.51   8.74      8.69   8.55    9.36  2.7027   
           000005.SZ       6.04   6.24      6.08   6.10    6.64  3.3113   
...                         ...    ...       ...    ...     ...     ...   
2023-12-29 688799.SH      47.21  48.94     47.69  47.22   51.93  3.6645   
           688800.SH      39.52  40.18     37.42  39.50   43.47  1.6700   
           688819.SH      27.91  27.92     27.02  28.00   30.70  0.0358   
           688981.SH      53.13  53.02     51.70  53.00   58.44 -0.2070   
           689009.SH      29.20  29.66     28.58  29.25   32.12  1.5753   

                        

In [29]:
merge_df.to_csv('处理过的全量数据.csv')